In [1]:
import os
import torch
from d2l import torch as d2l
import random
import math

In [2]:
inputfile = 'data/default/sent2vec/train_data/'
outputfile = 'data/default/sent2vec/vec_data/'

In [3]:
uc = 0
tc = 0  
ic = 0 
di = 0 
sent_item_list = list()
item_set = set()
click_list = list()
uoffs_list = list()
uid_list = list()
uoffs_list.append(0)
uid_list.append(0)
all_click_list = list()
with open(os.path.join(inputfile,'train.log')) as fp:
    for line in fp:
        line = line[:-1]
        line_info = line.split('\t')
        uid = line_info[0]
        md5_info = line_info[1:]
        for md5 in md5_info:
            item_set.add(md5)
            all_click_list.append(md5)
            click_list.append(md5)
            tc += 1
        sent_item_list.append(click_list)
        click_list = list()
        uc += 1 
        uoffs_list.append(tc)
        uid_list.append(uid)

In [4]:
len(sent_item_list) # 每个句子

1562291

In [5]:
len(uid_list) #所有点击item

1562292

In [6]:
len(item_set) #item 集合

29182

In [7]:
counter = d2l.count_corpus(sent_item_list)

建立词表

In [8]:
vocab = d2l.Vocab(sent_item_list)
f'vocab size: {len(vocab)}'

'vocab size: 29183'

In [9]:
corpus = [vocab[line] for line in sent_item_list]
corpus[:3]

[[539, 241, 241, 790],
 [1543],
 [20566,
  11494,
  13071,
  20566,
  23757,
  5991,
  1645,
  16316,
  4521,
  4497,
  1442,
  6132,
  3197,
  4732,
  1396,
  1659,
  1120]]

In [10]:
def get_centers_and_contexts(corpus):
    """返回跳元模型中的中心词和上下文词，sent2vec是句子"""
    centers, contexts = [], []
    for line in corpus:
        # 要形成“中心词-上下文词”对，每个句子一个词和剩余的所有此词形成
#         if len(line) < 2:
#             continue
        centers += line[:len(line)-1]
        for i in range(len(line)-1):  # 上下文窗口中间i
            indices = list(range(i,len(line)))
            # 从上下文词中排除中心词
            indices.remove(i)
            contexts.append([line[idx] for idx in indices])
    return centers, contexts

测试形成中心词和上下文词

In [11]:
tiny_dataset = [list(range(7)), list(range(7, 10))]
print('数据集', tiny_dataset)
for center, context in zip(*get_centers_and_contexts(tiny_dataset)):
    print('中心词', center, '的上下文词是', context)

数据集 [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
中心词 0 的上下文词是 [1, 2, 3, 4, 5, 6]
中心词 1 的上下文词是 [2, 3, 4, 5, 6]
中心词 2 的上下文词是 [3, 4, 5, 6]
中心词 3 的上下文词是 [4, 5, 6]
中心词 4 的上下文词是 [5, 6]
中心词 5 的上下文词是 [6]
中心词 7 的上下文词是 [8, 9]
中心词 8 的上下文词是 [9]


In [12]:
all_centers, all_contexts = get_centers_and_contexts(corpus)
f'# “中心词-上下文词对”的数量: {sum([len(contexts) for contexts in all_contexts])}'

'# “中心词-上下文词对”的数量: 234981454'

In [21]:
len(all_contexts)

8828074

In [13]:
len(all_centers) # 中心词有

8828074

负采样

In [14]:
class RandomGenerator:
    """根据n个采样权重在{1,...,n}中随机抽取"""
    def __init__(self, sampling_weights):
        # Exclude
        self.population = list(range(1, len(sampling_weights) + 1))
        self.sampling_weights = sampling_weights
        self.candidates = []
        self.i = 0

    def draw(self):
        if self.i == len(self.candidates):
            # 缓存k个随机采样结果
            self.candidates = random.choices(
                self.population, self.sampling_weights, k=10000)
            self.i = 0
        self.i += 1
        return self.candidates[self.i - 1]

In [22]:
all_contexts[0]

[241, 241, 790]

In [23]:
def get_negatives(all_contexts, vocab, counter, K):
    """返回负采样中的噪声词"""
    # 索引为1、2、...（索引0是词表中排除的未知标记）
    sampling_weights = [counter[vocab.to_tokens(i)]**0.75
                        for i in range(0, len(vocab))]
    all_negatives, generator = [], RandomGenerator(sampling_weights)
    for contexts in all_contexts:
        negatives = []
        while len(negatives) <  K:
            neg = generator.draw()
            # 噪声词不能是上下文词
            if neg not in contexts:
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

all_negatives = get_negatives(all_contexts, vocab, counter, 20)